In [ ]:
import socket
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend  # Make sure to import this
import ssl

def load_private_key():
    with open("private_key.pem", "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()  # Use the imported default_backend
        )
    return private_key

def load_public_key():
    with open("public_key.pem", "rb") as key_file:
        public_key = serialization.load_pem_public_key(
            key_file.read(),
            backend=default_backend()  # Use the imported default_backend
        )
    return public_key

def start_server():
    # 加载公钥
    public_key = load_public_key()

    # 创建套接字
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 65432))
    server_socket.listen(5)

    # 配置SSL
    context = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH)
    context.load_cert_chain(certfile='server.crt', keyfile='server.key')

    # 包装原始套接字为SSL套接字
    secure_socket = context.wrap_socket(server_socket, server_side=True)

    while True:
        # 接受客户端连接
        client_socket, addr = secure_socket.accept()
        print(f"Connected securely by {addr}")

        # 接收数据
        data = client_socket.recv(1024)
        if len(data) < 256:
            print("Error: Data received is too short to contain a valid signature.")
            continue
        
        message = data[:-256]  # 假设签名是最后256字节
        signature = data[-256:]

        # 验证签名
        is_valid = verify_signature(public_key, message, signature)
        print(f"Received message: {message.decode('utf-8')}")
        print(f"Signature valid: {is_valid}")

        client_socket.close()

def verify_signature(public_key, message, signature):
    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(
              mgf=padding.MGF1(hashes.SHA256()),
              salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        print(f"Verification failed: {e}")
        return False

if __name__ == "__main__":
    start_server()


Connected securely by ('127.0.0.1', 59643)
Received message: This is a test message from the cardholder.
Signature valid: True
